In [1]:
from face_extractor import Extractor

In [2]:
Extractor.extract(imagePath='test/Marilyn_Monroe.jpg',cascadeFile=Extractor.HAARCASCADE_ALT2,size=30)
Extractor.extract(imagePath='test/obama_cliton.jpg',cascadeFile=Extractor.HAARCASCADE_ALT2,size=30);

In [3]:
labels = open("classes.txt", "r").readlines()

In [4]:
from keras.models import model_from_json
from keras.preprocessing.image import *
json_file = open('model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights('model.h5')

Using CNTK backend


In [5]:
Marilyn_Monroe = img_to_array(load_img('test/Marilyn_Monroe_0.jpg'))/255
Marilyn_Monroe = Marilyn_Monroe.reshape((1,) + Marilyn_Monroe.shape) 

In [6]:
model.predict_classes(Marilyn_Monroe)

1/1 [==============================] - 0s


array([65], dtype=int64)

In [7]:
labels[65]

'marilyn monroe\n'

In [8]:
clinton = img_to_array(load_img('test/obama_cliton_0.jpg'))/255
clinton = clinton.reshape((1,) + clinton.shape) 

In [9]:
model.predict_classes(clinton)

1/1 [==============================] - 0s


array([35], dtype=int64)

In [10]:
labels[35]

'hillary clinton\n'

In [11]:
obama = img_to_array(load_img('test/obama_cliton_1.jpg'))/255
obama = obama.reshape((1,) + obama.shape) 

In [12]:
model.predict_classes(obama)

1/1 [==============================] - 0s


array([10], dtype=int64)

In [13]:
labels[10]

'barack obama\n'